In [136]:
# imports
import category_encoders as ce
import pandas as pd
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle

In [115]:
# load in dataset
df = pd.read_csv('MLB-Stats.csv')
print(df.shape)
df.head()

(28293, 28)


,playerID,Player Name,weight,height,bats,throws,Season,League,teamID,Team,...,RBI,SB,BB,SO,salary,IBB,HBP,SH,SF,PA
0,aardsda01,David Aardsma,215,75,R,R,2004,NL,SFN,San Francisco Giants,...,0,0,0,0,300000,0,0,0,0,0
1,aardsda01,David Aardsma,215,75,R,R,2007,AL,CHA,Chicago White Sox,...,0,0,0,0,387500,0,0,0,0,0
2,aardsda01,David Aardsma,215,75,R,R,2008,AL,BOS,Boston Red Sox,...,0,0,0,1,403250,0,0,0,0,1
3,aardsda01,David Aardsma,215,75,R,R,2009,AL,SEA,Seattle Mariners,...,0,0,0,0,419000,0,0,0,0,0
4,aardsda01,David Aardsma,215,75,R,R,2012,AL,NYA,New York Yankees,...,0,0,0,0,500000,0,0,0,0,0


In [116]:
# check for any missing data
df.isnull().sum()

playerID       0
Player Name    0
weight         0
height         0
bats           0
throws         0
Season         0
League         0
teamID         0
Team           0
Franchise      0
G              0
AB             0
R              0
H              0
2B             0
3B             0
HR             0
RBI            0
SB             0
BB             0
SO             0
salary         0
IBB            0
HBP            0
SH             0
SF             0
PA             0
dtype: int64

In [117]:
# data exploration
df.describe().T

,count,mean,std,min,25%,50%,75%,max
weight,28293.0,1.990493e+02,2.265398e+01,140.0,185.0,195.0,215.0,315.0
height,28293.0,7.350019e+01,2.286595e+00,66.0,72.0,74.0,75.0,83.0
Season,28293.0,2.000938e+03,8.900551e+00,1985.0,1994.0,2001.0,2009.0,2016.0
G,28293.0,6.393387e+01,4.728526e+01,1.0,26.0,50.0,101.0,163.0
AB,28293.0,1.618799e+02,1.994066e+02,0.0,1.0,57.0,298.0,716.0
R,28293.0,2.178684e+01,3.013390e+01,0.0,0.0,4.0,37.0,152.0
H,28293.0,4.262454e+01,5.620091e+01,0.0,0.0,10.0,76.0,262.0
2B,28293.0,8.220655e+00,1.140145e+01,0.0,0.0,2.0,14.0,59.0
3B,28293.0,9.091648e-01,1.832408e+00,0.0,0.0,0.0,1.0,23.0
HR,28293.0,4.776977e+00,8.384449e+00,0.0,0.0,0.0,6.0,73.0


In [118]:
# interesting stat ideas
# avg salary by year - shows growth in cap, inflation
# total salary per team by year
# how has this changed since moneyball, statistics used to determine best value players?

In [119]:
# change column names from abbreviations to full names to help with domain knowledge
df = df.rename(columns={'G': 'Games_Played', 'AB': 'At_bats', 'R': 'Runs', 'H': 'Hits',
                   '2B': 'Doubles', '3B': 'Triples', 'HR': 'Home_Runs', 'SB': 'Stolen_Base',
                   'BB': 'Walk', 'SO': 'Strikeouts', 'IBB': 'Intentional_Walk', 'HBP': 'Hit_By_Pitch',
                   'SH': 'Sacrifice_Bunt', 'SF': 'Sacrifice_Fly', 'PA': 'Plate_Appearance'})
df.columns

Index(['playerID', 'Player Name', 'weight', 'height', 'bats', 'throws',
       'Season', 'League', 'teamID', 'Team', 'Franchise', 'Games_Played',
       'At_bats', 'Runs', 'Hits', 'Doubles', 'Triples', 'Home_Runs', 'RBI',
       'Stolen_Base', 'Walk', 'Strikeouts', 'salary', 'Intentional_Walk',
       'Hit_By_Pitch', 'Sacrifice_Bunt', 'Sacrifice_Fly', 'Plate_Appearance'],
      dtype='object')

In [120]:
# how frequently a batter reaches base per plate appearance
# times on base include hits, walks and hit-by-pitches
df['On_Base_Percentage'] = (df['Hits'] + df['Walk'] + df['Hit_By_Pitch']) / df['Plate_Appearance']
df['On_Base_Percentage'] = df['On_Base_Percentage'].fillna(0)

In [121]:
# batting average - hits divided by total at-bats
df['AVG'] = df['Hits'] / df['At_bats']
df['AVG'] = df['AVG'].fillna(0)

In [122]:
# how stats are changing from season to season?
# the average weight of players has increased from 187 in 1985 to 212 in 2016
# the average salary of players has increased from 479k in 1985 to 4.49m in 2016
df.groupby('Season').mean()

,weight,height,Games_Played,At_bats,Runs,Hits,Doubles,Triples,Home_Runs,RBI,...,Walk,Strikeouts,salary,Intentional_Walk,Hit_By_Pitch,Sacrifice_Bunt,Sacrifice_Fly,Plate_Appearance,On_Base_Percentage,AVG
Season,,,,,,,,,,,,,,,,,,,,,
1985,187.022530,73.318891,77.838821,209.284229,27.003466,54.677643,9.636049,1.417678,5.532062,25.902946,...,20.611785,31.521664,4.799501e+05,2.029463,1.025997,2.091854,1.738302,236.781629,0.217251,0.173327
1986,187.722581,73.329032,69.200000,177.620645,23.233548,46.061935,8.147097,1.078710,4.846452,21.867097,...,17.794839,30.476129,4.180652e+05,1.621935,1.014194,1.840000,1.450323,201.341935,0.208224,0.165531
1987,187.481426,73.300149,70.114413,184.178306,25.619614,48.945022,8.750371,1.120357,5.912333,24.438336,...,18.870728,31.603269,4.315243e+05,1.786033,1.084695,1.763744,1.435364,209.118871,0.214620,0.174368
1988,187.520509,73.333805,69.878359,186.203678,22.768034,47.612447,8.435644,1.097595,4.256011,21.555870,...,17.260255,30.178218,4.613403e+05,1.850071,1.210750,2.052334,1.676096,210.253182,0.206076,0.164955
1989,187.988281,73.351562,68.520833,177.423177,21.764323,45.528646,7.893229,1.096354,3.899740,20.391927,...,16.884115,29.031250,5.043765e+05,1.835938,0.996094,1.972656,1.545573,200.657552,0.208437,0.168494
1990,187.771300,73.357623,59.614350,149.899103,18.848655,38.823991,6.903587,0.913677,3.489910,17.822870,...,14.523543,24.705157,5.296594e+05,1.467489,0.903587,1.627803,1.354260,169.775785,0.189931,0.154104
1991,188.782730,73.427577,68.036212,171.235376,21.883008,44.153203,7.922006,1.101671,4.186630,20.934540,...,16.891365,28.504178,8.866946e+05,1.543175,1.086351,1.917827,1.540390,194.214485,0.201263,0.158360
1992,189.168553,73.398742,66.166038,167.240252,20.522013,43.031447,7.801258,0.987421,3.664151,19.323270,...,16.277987,27.106918,1.078035e+06,1.615094,1.161006,1.835220,1.535849,189.665409,0.200073,0.161265
1993,190.160696,73.430911,60.757421,153.820880,20.761515,40.990788,7.401228,0.930399,4.038895,19.612078,...,15.035824,25.765609,9.904447e+05,1.507677,1.189355,1.788127,1.442170,174.784033,0.199621,0.164202


In [123]:
# TODO - get the sum of each teams salaries for each season
# this is only for hitters not for pitchers?
df.groupby(['Season', 'Team']).sum()

weight  height  Games_Played  At_bats  Runs  \
Season Team                                                                
1985   Atlanta Braves          3964    1540          1767     4551   513   
       Baltimore Orioles       4291    1679          1911     5163   783   
       Boston Red Sox          4511    1763          1818     5185   726   
       California Angels       4792    1904          1848     4826   646   
       Chicago Cubs            4312    1700          1880     4823   603   
...                             ...     ...           ...      ...   ...   
2016   St. Louis Cardinals     6150    2129          2155     5046   699   
       Tampa Bay Rays          6767    2355          1882     4722   596   
       Texas Rangers           6816    2356          1770     4492   645   
       Toronto Blue Jays       7542    2650          2227     5398   747   
       Washington Nationals    5977    2005          2021     4992   683   

                             Hits  Doubles  Triples  Home_Runs  RBI  ...  \
Season Team                                                          ...   
1985   Atlanta Braves        1131      179       21        119  540  ...   
       Baltimore Orioles     1389      226       22        207  747  ...   
       Boston Red Sox        1478      270       28        151  712  ...   
       California Angels     1210      192       27        133  608  ...   
       Chicago Cubs          1252      213       23        141  588  ...   
...                           ...      ...      ...        ...  ...  ...   
2016   St. Louis Cardinals   1272      267       29        208  675  ...   
       Tampa Bay Rays        1167      250       30        197  570  ...   
       Texas Rangers         1189      227       17        184  630  ...   
       Toronto Blue Jays     1348      271       18        220  725  ...   
       Washington Nationals  1252      244       20        185  674  ...   

                             Walk  Strikeouts     salary  Intentional_Walk  \
Season Team                                                                  
1985   Atlanta Braves         452         659   14032000                50   
       Baltimore Orioles      574         847   12073212                30   
       Boston Red Sox         517         726   10762227                38   
       California Angels      608         810   13191227                47   
       Chicago Cubs           524         812   13227917                58   
...                           ...         ...        ...               ...   
2016   St. Louis Cardinals    481        1239  136369500                22   
       Tampa Bay Rays         400        1250   66454510                18   
       Texas Rangers          350         974  180190323                22   
       Toronto Blue Jays      626        1331  191463367                16   
       Washington Nationals   502        1140  153733246                48   

                             Hit_By_Pitch  Sacrifice_Bunt  Sacrifice_Fly  \
Season Team                                                                
1985   Atlanta Braves                  18              41             37   
       Baltimore Orioles               17              28             38   
       Boston Red Sox                  29              41             53   
       California Angels               36              89             32   
       Chicago Cubs                    13              56             34   
...                                   ...             ...            ...   
2016   St. Louis Cardinals             62              32             35   
       Tampa Bay Rays                  63               9             24   
       Texas Rangers                   59              12             30   
       Toronto Blue Jays               53              26             40   
       Washington Nationals            61              43             61   

                             Plate_Appearance  

In [124]:
# TODO - predict the salary for a player based on their stats

In [125]:
# baseline
# how are we going to deal with a 10x increase in avg salary overtime?
# is there a way we could account for inflation?
# or could we just look one year

In [126]:
# average salary by year
avg_salary = pd.DataFrame(df['salary'].groupby(df['Season']).mean().astype(int))
avg_salary.columns = ['Avg_Salary']
avg_salary.head()

,Avg_Salary
Season,
1985,479950
1986,418065
1987,431524
1988,461340
1989,504376


In [127]:
df = df.merge(avg_salary, how='inner', on='Season')

In [128]:
# mean absolute error
errors = df['Avg_Salary'] - df['salary']
mean_absolute_error = errors.abs().mean()
print(f"""If we just guessed a player's salary to be the average salary of a player
in that season it would be off by an average of ${mean_absolute_error:,.0f}""")

If we just guessed a player's salary to be the average salary of a player
in that season it would be off by an average of $2,031,353


In [129]:
df.columns

Index(['playerID', 'Player Name', 'weight', 'height', 'bats', 'throws',
       'Season', 'League', 'teamID', 'Team', 'Franchise', 'Games_Played',
       'At_bats', 'Runs', 'Hits', 'Doubles', 'Triples', 'Home_Runs', 'RBI',
       'Stolen_Base', 'Walk', 'Strikeouts', 'salary', 'Intentional_Walk',
       'Hit_By_Pitch', 'Sacrifice_Bunt', 'Sacrifice_Fly', 'Plate_Appearance',
       'On_Base_Percentage', 'AVG', 'Avg_Salary'],
      dtype='object')

In [130]:
# features
features = [
 'weight',
 'height',
 'Season',
 'League',
 'teamID',
 'Games_Played',
 'At_bats',
 'Runs',
 'Hits',
 'Doubles',
 'Triples',
 'Home_Runs',
 'RBI',
 'Stolen_Base',
 'Walk',
 'Strikeouts',
 'Intentional_Walk',
 'Hit_By_Pitch',
 'Sacrifice_Bunt',
 'Sacrifice_Fly',
 'Plate_Appearance',
 'On_Base_Percentage',
 'AVG']

# shuffle data
df = shuffle(df)
# drop target
y = df.pop('salary')
X = df

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((16975, 30), (5659, 30), (5659, 30), (16975,), (5659,), (5659,))

In [131]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    DecisionTreeRegressor())

# Fit on train
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print('Mean Absolute Error', metrics.mean_absolute_error(y_test, y_pred))

Mean Absolute Error 1579799.7997879484


In [133]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    KNeighborsRegressor())

# Fit on train
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print('Mean Absolute Error', metrics.mean_absolute_error(y_test, y_pred))

Mean Absolute Error 1903506.6880014136


In [137]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    RandomForestRegressor())

# Fit on train
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print('Mean Absolute Error', metrics.mean_absolute_error(y_test, y_pred))

/Users/josephbell/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Mean Absolute Error 1315363.4369667366
